## lmd - prepare synthetic dataset (Alpaca template) & fine-tune w/ unsloth

> 2024/03/24 : update w/ mistral v0.2

**Steps / end goal**
- 1. Started with our +-500 human annotated comments (out of 200k)
- 2. Synthetic data generation (comments + label) w/ Mistral OpenHermes : around 2k samples
- 3. Prepare instruction dataset, before fine tuning, using Alpaca format.**<- we're here**    
- 4. Fine-tune mistral-7B (classif. / label completion), using unsloth, on train + synthetic data.**<- we're here**  
- 5. More tests on the fine-tuned model. If good enough, labels unlabeled data to several k examples (fine-tuned model as a classifier or weighted avg. w/ our Few shot SetFit baseline).
- 6. Extend dataset to several 20k examples with fine-tuned Mistral (and/or ensemble model w/ Setfit) doing the classification.  
- 7. End goal being deployment/inference performance: train a classifier on the extended dataset using bge-m3 or multi-e5 embeddings. 

**Ressources**  
- [MLabonne Repo](https://github.com/mlabonne/llm-course)  
- [Dataset Gen - Kaggle example](https://www.kaggle.com/code/phanisrikanth/generate-synthetic-essays-with-mistral-7b-instruct)  
- [Dataset Gen - blog w/ prompt examples](https://hendrik.works/blog/leveraging-underrepresented-data)  
- [Prepare dataset- /r/LocalLLaMA best practice classi](https://www.reddit.com/r/LocalLLaMA/comments/173o5dv/comment/k448ye1/?utm_source=reddit&utm_medium=web2x&context=3)  
- [Prepare dataset - using gpt3.5](https://medium.com/@kshitiz.sahay26/how-i-created-an-instruction-dataset-using-gpt-3-5-to-fine-tune-llama-2-for-news-classification-ed02fe41c81f) 
- [PreparePredibase prompts for diverse fine-tuning tasks](https://predibase.com/lora-land)
- [Fine tune OpenHermes-2.5-Mistral-7B - including prompt template gen](https://towardsdatascience.com/fine-tune-a-mistral-7b-model-with-direct-preference-optimization-708042745aac)  
- [Fine tune - Unsloth colab example](https://colab.research.google.com/drive/1Dyauq4kTZoLewQ1cApceUQVNcnnNTzg_?usp=sharing)
- [Fine tune - w/o unsloth](https://gathnex.medium.com/mistral-7b-fine-tuning-a-step-by-step-guide-52122cdbeca8) or this one by [wandb](https://wandb.ai/vincenttu/finetuning_mistral7b/reports/Fine-tuning-Mistral-7B-with-W-B--Vmlldzo1NTc3MjMy)  

Beware installation might take a few minutes

In [1]:
#%%capture
# cf. colab example in ressources for installation / run in notebook
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

# Only if running in kaggle, prevents trl ImportError. Also, you do need to restart the kernel.
%pip install -U datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-oyw6b56q/unsloth_0edf24dc43534b65bd3ef7d80d4844e0
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-oyw6b56q/unsloth_0edf24dc43534b65bd3ef7d80d4844e0
  Resolved https://github.com/unslothai/unsloth.git to commit a100afa60cf1566e784b41eb462b5d2ed80977f8
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... - \ done
  Installing backend dependencies ... - \ done
  Preparing metadata (pyproject.toml) ... - \ done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 

In [2]:
import warnings
warnings.filterwarnings('ignore')
import os

import pandas as pd

import torch
from unsloth import FastLanguageModel

from datasets import load_dataset, Dataset, DatasetDict
from trl import SFTTrainer
from transformers import TrainingArguments

2024-03-24 13:34:13.776564: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 13:34:13.776663: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 13:34:13.963544: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Wandb monitoring + loggin to HF to upload model :

In [3]:
%%capture
# wandb login to monitor fine-tuning run(s)
import wandb
from kaggle_secrets import UserSecretsClient

# wandb
user_secrets = UserSecretsClient()
wandb_token = user_secrets.get_secret("wandb_ft")
os.environ["WANDB_PROJECT"]= "mistral_ft"
wandb.login(key=wandb_token)

# HuggingFace to save model / dataset
hf_token = user_secrets.get_secret("hf_key")

### Load datasets : synthetic data & human labeled dataset

Reminder dataset(s) structure :  
- Original data :  
`train` : around 300 labeled comments (0. pro_ukraine, 1. pro_russia, 2. other_topic)  
`validation` : our spare (120) labeled comments, used for benchmark    
`unlabeled` : rest of comments, no label. Maybe labeled later w/ fine-tuned classification model  
- Synthetic data :  
Around 1,2k synthetic comments, generated w/ Mistral, from original data/`train` seed.

In [4]:
%%capture
# load original datased (saved on HF) and synthetic datasets (saved locally)
filepath_labeled_data = "gentilrenard/lmd_ukraine_comments"
filepath_synthetic_data = "/kaggle/input/lmd-synthetic/lmd_synthetic_data.parquet"

# load original, custom and human-annotated dataset
original_dataset = load_dataset(filepath_labeled_data)

# load generated data
synthetic_dataset = pd.read_parquet(filepath_synthetic_data)

### Concatenate synthetic and labeled datasets

Build our full dataset (300 labeled train + 2k synthetic, 139 labeled test), to be later loaded as a "datasets", generate all our instructions prompts & fine-tune. We're keeping original dataset "evaluation" set for

In [5]:
# Concatenate train and synthetic data : 2520 rows
# Column 'text' is renamed to 'comment'. A new col "text" will be created later when building prompts
dataset_train = original_dataset['train'].to_pandas()
dataset_full = pd.concat([dataset_train, synthetic_dataset])
dataset_full = dataset_full.rename(columns={"text":"comment"})

# extract "validation" from labeled dataset, this will be our test data
dataset_test = original_dataset['validation'].to_pandas()
dataset_test = dataset_test.rename(columns={"text":"comment"})

# created a new HF DatasetDict : train with 2520 labeled data, and test with our 139 labeled samples
dataset = DatasetDict({
    'train': Dataset.from_pandas(dataset_full),
    'test': Dataset.from_pandas(dataset_test),
})

# remove extra column added at creation
dataset['train'] = dataset['train'].remove_columns(['__index_level_0__'])

print(dataset)
# TODO, we might upload it to HF later.

DatasetDict({
    train: Dataset({
        features: ['comment', 'label'],
        num_rows: 2529
    })
    test: Dataset({
        features: ['comment', 'label'],
        num_rows: 139
    })
})


### Load model, tokenizer (Unsloth `FastLanguageModel`)

Cf. [HF](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth) documentation or [Unsloth repo](https://github.com/unslothai/unsloth/tree/main#-documentation). Basically we replace `AutoModelForCausalLM` with Unsloth `FastLanguageModel`

In [6]:
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # True to use 4bit quantization / reduce memory usage (for T4 GPU)

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    #model_name = "mistralai/Mistral-7B-v0.1", #"unsloth/mistral-7b-bnb-4bit",
    model_name = "unsloth/mistral-7b-v0.2-bnb-4bit", # update to 0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


### Add LoRa adapters

In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # can improve fine-tuning, at attention/feed fwd layers
    target_modules = [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha = 16, # more change to pre-train weights but care overfitting
    lora_dropout = 0.05, # any, but 0 if perf opti.
    bias = "none",    # any, but "none" is perf  opti.
    use_gradient_checkpointing = True,
    random_state = 11,
    use_rslora = False,  # support rank stabilized LoRA
    loftq_config = None, # LoftQ support
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.3 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


### Data Preparation (Alpaca template)

In [9]:
prompt = """You are a helpful, precise, detailed, and concise artificial intelligence assistant with a deep expertise in reading and interpreting comments about Ukraine invasion by Russia.
In February 2022, Russia's invasion of Ukraine escalated the ongoing conflict in Ukraine Dombass region since 2014 and is causing massive casualties. President Putin claimed the operation aimed to "demilitarize and denazify" Ukraine. Despite Russian territorial gains, Ukraine's resistance and counterattacks have reclaimed some areas. The international community responded with sanctions, support for Ukraine, and legal actions against Russia.
You are very intelligent and sharp, having a keen ability and nuanced enough to distinguish which side of the conflict the comment is on.
Your task is to classify a comment into one of 3 labels : 0, 1 or 2. Possible labels and their meaning:
- 0: rather in favor of Ukraine and its allies. Support sanctions against Russia or criticizes Russian policy. Ukraine will win.
- 1: in favor of Russia, even if only so slightly. Criticizes Ukraine, Western, UE or OTAN policies against Russia. Fears of a costly escalation if support is brought to Ukraine. Russia will win.
- 2: irrelevant to the conflict or does not take side.
You will be evaluated based on the following criteria: - The generated answer is best matching label for the comment. - The generated answer is always one label (0, 1 or 2).
Categorize the comment into a single comment label only:
### Comment:
{}
### Comment label:
{}"""

In [10]:
# Add EOS special token, according to previously loaded tokenizer
EOS_TOKEN = tokenizer.eos_token
def format_prompts(examples):
    inputs = examples["comment"]
    outputs = examples["label"]
    texts = []
    for inp, output in zip(inputs, outputs):
        # Add end of string token to prevent infinite generations.
        text = prompt.format(inp, output) + EOS_TOKEN
        texts.append(text)
    return {"text":texts,}

In [11]:
# Build prompts
dataset['train'] = dataset['train'].map(format_prompts, batched = True,)

# Print a sample :
dataset['train'][3]['text']

Map:   0%|          | 0/2529 [00:00<?, ? examples/s]

'You are a helpful, precise, detailed, and concise artificial intelligence assistant with a deep expertise in reading and interpreting comments about Ukraine invasion by Russia.\nIn February 2022, Russia\'s invasion of Ukraine escalated the ongoing conflict in Ukraine Dombass region since 2014 and is causing massive casualties. President Putin claimed the operation aimed to "demilitarize and denazify" Ukraine. Despite Russian territorial gains, Ukraine\'s resistance and counterattacks have reclaimed some areas. The international community responded with sanctions, support for Ukraine, and legal actions against Russia.\nYou are very intelligent and sharp, having a keen ability and nuanced enough to distinguish which side of the conflict the comment is on.\nYour task is to classify a comment into one of 3 labels : 0, 1 or 2. Possible labels and their meaning:\n- 0: rather in favor of Ukraine and its allies. Support sanctions against Russia or criticizes Russian policy. Ukraine will win.\

### Train the model

Using HF, trl / Supervised Fine-tuning (SFT) Trainer  
Post training remarks :  
- epoch 1 was more than enough, train loss plateau around 60 steps only
- training time w/ unsloth + mistral-7B (4bit): 1h for 1 epoch, 20mn for 100 steps.


In [12]:
training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 5,
    num_train_epochs = 1,
    # max_steps = 110,
    learning_rate = 2e-4, # 2e-5
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    optim = "adamw_8bit",
    weight_decay = 0.01,
    lr_scheduler_type = "linear",
    seed = 11,
    output_dir = "outputs",
    run_name = "mistral-7b-base_en_v12",
    logging_strategy = 'steps',
    logging_steps = 1,
    save_strategy="steps",
    save_steps=10,
    save_total_limit = 2,
    report_to = "wandb",
    )

In [13]:
# init trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2, # if packing = False, else default to None
    packing = False, # more speed if packing short sequences. Maybe later
    args = training_args,
)

Map (num_proc=2):   0%|          | 0/2529 [00:00<?, ? examples/s]

In [14]:
# train
trainer = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,529 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 316
 "-____-"     Number of trainable parameters = 41,943,040
wandb: Currently logged in as: vionmatthieu. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.2
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240324_133510-c6h2mddr
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run mistral-7b-base_en_v12
wandb: ⭐️ View project at https://wandb.ai/vionmatthieu/mistral_ft
wandb: 🚀 View run at https://wandb.ai/vionmatthieu/mistral_ft/runs/c6h2mddr


Step,Training Loss
1,2.253900
2,2.349900
3,2.180100
4,1.958200
5,1.684800
6,1.365500
7,1.200600
8,0.855400
9,0.548200
10,0.398800


### Inference

Run the model we just trained. With Unsloth `FastLanguageModel`, test on one sample.

In [15]:
print(dataset['test'][22])
comment = dataset['test'][22]['comment']
print(comment)

{'comment': "Manque juste les pipelines... c'est effarant de ne pas y avoir pensé ! Y'a-t-il un Capitaine dans le sous-marin ?", 'label': 2}
Manque juste les pipelines... c'est effarant de ne pas y avoir pensé ! Y'a-t-il un Capitaine dans le sous-marin ?


In [16]:
# (params and prompt format defined earlier)
FastLanguageModel.for_inference(model) # Unsloth inference

inputs = tokenizer(
[
    prompt.format(
        comment, # insert comment
        "", # output - blank (instead of label) for generation
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 3, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> You are a helpful, precise, detailed, and concise artificial intelligence assistant with a deep expertise in reading and interpreting comments about Ukraine invasion by Russia.\nIn February 2022, Russia\'s invasion of Ukraine escalated the ongoing conflict in Ukraine Dombass region since 2014 and is causing massive casualties. President Putin claimed the operation aimed to "demilitarize and denazify" Ukraine. Despite Russian territorial gains, Ukraine\'s resistance and counterattacks have reclaimed some areas. The international community responded with sanctions, support for Ukraine, and legal actions against Russia.\nYou are very intelligent and sharp, having a keen ability and nuanced enough to distinguish which side of the conflict the comment is on.\nYour task is to classify a comment into one of 3 labels : 0, 1 or 2. Possible labels and their meaning:\n- 0: rather in favor of Ukraine and its allies. Support sanctions against Russia or criticizes Russian policy. Ukraine will 

### Save the model (locally + HuggingFace)

Save LoRA adapters or a merged version (model + adapters), using Unsloth.  
For an alternative approach see this blog [post](https://towardsdatascience.com/fine-tune-a-mistral-7b-model-with-direct-preference-optimization-708042745aac). (save artifacts, + load base model, tokenizer, merge with adapters using PeftModel etc., then save).

In [17]:
# !rm -rf /kaggle/working/*

In [18]:
# uncomment to save fully merged model (not only adapters)

# local_filename = "Mistral-7B-lora-lmd-en"
# hf_filename = "gentilrenard/Mistral-7B-lora-lmd-en"

# save merged locally
# model.save_pretrained_merged(local_filename, tokenizer, save_method = "merged_16bit",)
# push merged to HF Hub
# model.push_to_hub_merged(hf_filename, tokenizer, save_method = "merged_16bit", token = hf_token)

In [19]:
lora_local_filename = "Mistral-7B-lora-lmd-en-v3"
lora_hf_filename = "gentilrenard/Mistral-7B-lora-lmd-en-v3"

In [20]:
# save LoRA adapters
model.save_pretrained(lora_local_filename)
model.push_to_hub(lora_hf_filename, tokenizer, token = hf_token)

README.md:   0%|          | 0.00/593 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Saved model to https://huggingface.co/gentilrenard/Mistral-7B-lora-lmd-en-v3


### Load LoRA adapters & infer (`unsloth`)

In [21]:
print(f"Comment + label :\n{dataset['test'][22]}")
comment = dataset['test'][22]['comment']
print(f"Comment :\n{comment}")

Comment + label :
{'comment': "Manque juste les pipelines... c'est effarant de ne pas y avoir pensé ! Y'a-t-il un Capitaine dans le sous-marin ?", 'label': 2}
Comment :
Manque juste les pipelines... c'est effarant de ne pas y avoir pensé ! Y'a-t-il un Capitaine dans le sous-marin ?


#### One sample

In [22]:
# load LoRa adapters from HuggingFace
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = lora_hf_filename,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)

adapter_config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [23]:
inputs = tokenizer(
    [
        prompt.format(
        comment, # insert comment
        "", # output - blank (instead of label) for generation
        )
    ],
    return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> You are a helpful, precise, detailed, and concise artificial intelligence assistant with a deep expertise in reading and interpreting comments about Ukraine invasion by Russia.\nIn February 2022, Russia\'s invasion of Ukraine escalated the ongoing conflict in Ukraine Dombass region since 2014 and is causing massive casualties. President Putin claimed the operation aimed to "demilitarize and denazify" Ukraine. Despite Russian territorial gains, Ukraine\'s resistance and counterattacks have reclaimed some areas. The international community responded with sanctions, support for Ukraine, and legal actions against Russia.\nYou are very intelligent and sharp, having a keen ability and nuanced enough to distinguish which side of the conflict the comment is on.\nYour task is to classify a comment into one of 3 labels : 0, 1 or 2. Possible labels and their meaning:\n- 0: rather in favor of Ukraine and its allies. Support sanctions against Russia or criticizes Russian policy. Ukraine will 

#### Many samples

In [24]:
import re

for i, sample in enumerate(dataset['test']):
    inputs = tokenizer(
        [
            prompt.format(
            sample['comment'], # insert comment
            "", # output - blank (instead of label) for generation
            )
        ],
        return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 20, use_cache = True)
    decoded_outputs = tokenizer.batch_decode(outputs)
    
    # extract "comment label" in response
    matches = re.findall(r'Comment label:\n(\d)', decoded_outputs[0])
    predicted_label = int(matches[-1]) if matches else decoded_outputs

    print(f"Comment {i}:\n{sample['comment']}\nTrue label: {sample['label']}\nPredicted label:\n{predicted_label}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Comment 0:
Ça fait déjà 6 ans que Macron exprime son art du « en même temps » .... Vous n’avez toujours pas compris que cela ne veut rien dire ? Dire une chose et son contraire équivaut à ne parler pour rien ...
True label: 2
Predicted label:
2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Comment 1:
La question qui vaille et qui n'est pas posée dans cet article : celle des sources. En l'occurence, la couverture de cette guerre par la presse occidentale se fonde que sur une : les autorités ukrainiennes conseillées par une officine américaine liée à la CIA. Il doit en être de même mais acontrario pour la presse russe, mais c'est l'affaire des Russes. En revanche, la couverture par notre presse est nôtre affaire. Or, sans le moindre soupçon de doute, elle prend pour argent comptant tout ce que diffuse la propagande ukrainienne et otanesque. Pas un chiffre concernant les pertes humaines ou matérielles n'est crébible. Si d'aventure ils étaient réels, l'armée ukrainienne aurait réalisé un exploit plus qu'extraordinaire, à nul autre pareil dans l'histoire des batailles. En Ukraine, l'approche du front est interdite à tout journaliste à moins de 30km. La couverture de cette guerre est totalement dépendante de ce que le gouvernement ukrainien distille en aff

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Comment 135:
On continue d'utiliser le gaz Russe, mais on refuse de payer en Rouble car cela n'est pas conforme au droit. Mais une question intéressante sur le fond est : toutes nos sanctions et la prise (temporaire) de ce tableau sont-elles conformes au droit. Dis un peu autrement, le droit est-il toujours de notre côté dans cette affaire ? En effet, la France et l'UE ne sont pas en guerre. De quel droit avons nous le droit de faire tout cela ?
True label: 1
Predicted label:
1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Comment 136:
@mmalice il y a aussi bcp d'enfants qui ne sont pas des nouveaux nes, et qui sont places en familles d'acceuil. Que ne vous portez vous pas volontaire C'est un peu comme les gens qui veulent acheter des chatons ds les animaleries (entretenant un commerce contre le bien animal) et ne vont pas aider a prendre un animal plus age en refuge.... Desolee pour la comparaison, mais c'est le mm concept. S'il y avait si peu d'enfants a prendre en charges, les enfants plus grands et ados ne serait pas trimballes de famille en famille, de foyer en foyer
True label: 2
Predicted label:
2


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Comment 137:
@Capitalisme et machin truc votre message bidon, posté 20 fois dans ce fil de commentaires, à été « débunké » à plusieurs reprises de manière incontestable. Veuillez donc cesser de répandre sciemment de fausses informations qui de toutes façons ne convainc personne ici.
True label: 2
Predicted label:
2
Comment 138:
Je me demande si Fillon et Sarkozy sont toujours salariés par Poutine ?
True label: 0
Predicted label:
2
